In [1]:
pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org pulp

  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
Using cached PuLP-2.9.0-py3-none-any.whl (17.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\BANSAL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

plants = [1, 2, 3]
dcs = list(range(1, 13))

cap = {1: 12, 2: 9, 3: 10}
prod_cost = {1: 8, 2: 7, 3: 9}

demand = {1: 2, 2: 3, 3: 4, 4: 3.5, 5: 1.5, 6: 1.9, 7: 2.5, 8: 2, 9: 1.8, 10: 1.2, 11: 3, 12: 4}

trans_cost = {
    1: {1: 2, 2: 3, 3: 2, 4: 3, 5: 5, 6: 4.4, 7: 3.2, 8: 3, 9: 1.5, 10: 3, 11: 6.0, 12: 1.0},
    2: {1: 1.9, 2: 3, 3: 2.4, 4: 2.4, 5: 2, 6: 4.1, 7: 2.8, 8: 3, 9: 1.7, 10: 2, 11: 1.0, 12: 4.5},
    3: {1: 3.1, 2: 1.8, 3: 2.8, 4: 3.4, 5: 2, 6: 2.9, 7: 2.2, 8: 3.2, 9: 2, 10: 2.8, 11: 4.5, 12: 6.0}
}

model = LpProblem(name="Maximize_Profit", sense=LpMaximize)

x = {i: LpVariable(f"x{i}", lowBound=0) for i in plants}
y = {(i, j): LpVariable(f"y{i}_{j}", lowBound=0) for i in plants for j in dcs}

model += lpSum((20 - trans_cost[i][j]) * y[i, j] * 10000 for i in plants for j in dcs) - lpSum(prod_cost[i] * x[i] * 10000 for i in plants)

for i in plants:
    model += x[i] <= cap[i]

for i in plants:
    model += lpSum(y[i, j] for j in dcs) <= x[i]

for j in dcs:
    model += lpSum(y[i, j] for i in plants) == demand[j]

model.solve()

print(f"{'Plant':<10}{'Production (10,000 units)':<25}")
for i in plants:
    print(f"{i:<10}{x[i].varValue:<25}")

print(f"\n{'From Plant':<12}{'To DC':<8}{'Transported (10,000 units)':<25}")
for i in plants:
    for j in dcs:
        if y[i, j].varValue > 0:
            print(f"{i:<12}{j:<8}{y[i, j].varValue:<25}")

print(f"\nMax Profit: {model.objective.value():,.2f} Rupees")


Plant     Production (10,000 units)
1         12.0                     
2         9.0                      
3         9.4                      

From Plant  To DC   Transported (10,000 units)
1           1       0.7                      
1           3       4.0                      
1           8       1.5                      
1           9       1.8                      
1           12      4.0                      
2           1       1.3                      
2           4       3.5                      
2           10      1.2                      
2           11      3.0                      
3           2       3.0                      
3           5       1.5                      
3           6       1.9                      
3           7       2.5                      
3           8       0.5                      

Max Profit: 3,065,200.00 Rupees
